In [ ]:
def distributed_conv2d_embedding_model(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=20,\
                     kernel_size=(1, 8),\
                     strides=1,\
                     padding='same',\
                     name='conv_1',\
                     input_shape=input_shape))
    model.add(BatchNormalization())
    
    model.add(Conv2D(filters=20,\
                     kernel_size=(1, 4),\
                     strides=1,\
                     padding='same',\
                     name='conv_2',\
                     input_shape=input_shape))
    model.add(BatchNormalization())
    
    model.add(Activation('relu', name='relu_1'))
    model.add(MaxPooling2D(pool_size=(1, 2)))
    
    model.add(Flatten())
    return model

In [ ]:
def convlstm_embedding_model(input_shape):
    model = Sequential()
    model.add(ConvLSTM2D(filters=32, kernel_size=(1, 8),\
                         input_shape=input_shape,\
                         padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=32, kernel_size=(1, 4),
                       padding='same', return_sequences=True))
    model.add(BatchNormalization())
    model.add(Activation('relu', name='relu_1'))
    #model.add(MaxPooling3D(pool_size=(32, 16, 4)))
    return model

**Distributed CNN Model**

In [ ]:
windows, window_dim = 32, 16
input_shape = (windows, window_dim, 1)

#This model will take as input a 5D tensor of shape (batch_shape, timesequence/nb_frames, nb_row, nb_col, nb_channel)
#input shape: nb_frames, nb_row, nb_col, nb_channel
base_model = distributed_conv2d_embedding_model(input_shape)

#Added for BatchNormalization. See https://github.com/keras-team/keras/issues/2310
from keras import backend as K
K.set_learning_phase(1)

inputs = Input(shape=(None, 512), name='distributed_input')
y = Reshape((-1,) + input_shape, input_shape=(None, 512))(inputs)

y = TimeDistributed(base_model)(y)
y = LSTM(32, return_sequences=False)(y)
#y = Dense(30, activation='relu', name='counting_dense_1')(y)
outputs = Dense(1, kernel_initializer='normal', name='counting_dense_2')(y)
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
H = model.fit_generator(train_gen, validation_data=val_gen, validation_steps=len(val_gen),\
                        steps_per_epoch=n_bins, epochs=epochs)
test_loss, test_mae = model.evaluate_generator(test_gen, steps=len(test_gen))

**ConvLSTM Model**

In [ ]:
windows, window_dim = 32, 16
input_shape = (None, windows, window_dim, 1)

model = convlstm_embedding_model(input_shape)
H, H_test = lstm_counting_model(model, train_gen, val_gen, test_gen, n_bins)